Load API Keys

In [ ]:
from dotenv import load_dotenv
load_dotenv()

Load Contents from HLB Fixed Deposite Pages

In [ ]:
from langchain.document_loaders import WebBaseLoader

loader = WebBaseLoader([
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit.html?icp=hlb-en-all-footer-txt-fd",
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit/fixed-deposit-account/fixed-deposit-account.html", 
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit/fixed-deposit-account/e-fixed-deposit.html",
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit/fixed-deposit-account/flexi-fd.html",
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit/fixed-deposit-account/senior-savers-flexi-fd.html",
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit/fixed-deposit-account/junior-fixed-deposit.html",
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit/fixed-deposit-account/foreign-fixed-deposit-account.html",
  "https://www.hlb.com.my/en/personal-banking/help-support/fees-and-charges/deposits.html",
])
data = loader.load()
data

Split the Loaded Data into Chunks

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
texts = text_splitter.split_documents(data)
texts

Initialize Pinecone

In [ ]:
import pinecone
import os

pinecone.init(
  api_key=os.getenv("PINECONE_API_KEY"),
  environment=os.getenv("PINECONE_ENVIRONMENT")
)
index_name = "main-index"
namespace = "hlb-fixed-deposit"
if index_name not in pinecone.list_indexes():
  pinecone.create_index(
    name=index_name,
    metric="cosine",
    dimension=1536
  )
index = pinecone.Index(index_name)
index.delete(delete_all=True, namespace=namespace)

Create Embeddings and Vectors to Insert to Pinecone Database

In [ ]:
from langchain.vectorstores import Pinecone
from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
docs = [t.page_content for t in texts]
vector = Pinecone.from_texts(docs, embeddings, index_name=index_name, namespace=namespace)

Create Retriever for Vector Search

In [ ]:
retriever = vector.as_retriever(search_type="similarity_score_threshold" ,search_kwargs={'score_threshold': 0.6, 'k': 4})
retriever

Creating Prompt Template

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(model="gpt-4-1106-preview", temperature=0)
prompt_template = """
  You are a helpful assistant who should reply to inputs related only to the HongLeong Bank aka HLB.

  Chat History:
  {chat_history}

  Reply to the following input: {input}
  By only referring the follwing document: {document}
  You can refer to chat history also.
  If the document is null, you say you don't have information.
  If you feel like you don't have enough information, you say you don't have information.
  If you are not sure about deposit type, you can ask for clarification.
  Your answer should be detailed but if you have to do calculation, don't show your work or equations, just provide the final value, and you don't need to show reference.
  If the bank's name is not provided in the input, answer with the assumption that the question is about HLB.
  If the bank's name is provided and it's not HLB, don't answer that question.
  You don't need to answer if the input requires you to do something creative such as writing a song or making a joke.
"""
prompt = PromptTemplate(template=prompt_template, input_variables=['input', 'document', 'chat_history'])

Storing History and Creating Chain

In [ ]:
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferWindowMemory
from langchain.callbacks import get_openai_callback

memory = ConversationBufferWindowMemory(memory_key="chat_history", input_key="input", k=4)
chain = LLMChain(llm=llm, prompt=prompt, memory=memory, verbose=True)

def ask_question(question):
  with get_openai_callback() as cb:
      document = retriever.get_relevant_documents(question)
      result = chain.run(input=question, document=document)
      print(f'{cb}\n')
      print('Retrieved Documents:')
      for i in range(len(document)):
        print(f'{i}. {document[i]}')
      print(f'\nAnswer: {result}')


Running the Chain

In [ ]:
ask_question("What is compound interest?")

In [ ]:
ask_question("What is fixed deposit?")

In [ ]:
ask_question("How many types of fixed deposit are there?")

In [ ]:
ask_question("tell me about the 4th one")

In [ ]:
ask_question("What are the pros and cons of fixed deposit?")

In [ ]:
ask_question("Who is jay z?")

In [ ]:
ask_question("Are you sure you dont know Jay Z? He is a very popular hip hop artist?")

In [ ]:
ask_question("How many types of fixed deposit Citibank provide?")

In [ ]:
ask_question("What are the interest rates for Fixed Deposit?")

In [ ]:
ask_question("Can you find out by browsing the web?")

In [ ]:
ask_question("Lets say I want to invest RM 5000 in Fixed Deposit for 36 months and the interest rate is 5%. Please calculate total withdrawal amount at the end of the term. Take note the interest rate is compund interest?")

In [ ]:
ask_question("Lets say I want to invest RM 5000 in Fixed Deposit for 9 months. Please calculate total withdrawal amount at the end of the term.")

In [ ]:
ask_question("Can a non-resident of Malaysia invest in CIMB SGD fixed deposit?")

In [ ]:
ask_question("Who is the CEO of CIMB bank?")

In [ ]:
ask_question("What is CIMB net profit for the last financial year?")

In [ ]:
ask_question("What if Jay Z from United States wants to invest in SGD fixed deposit. Is that ok ?")

In [ ]:
ask_question("What if Michael Jackson wants to invest in SGD Fixed Deposit. Is that ok?")

In [ ]:
ask_question("Write a poem for HLB fixed deposit.")

In [ ]:
ask_question("What is the difference between fixed deposit and e-fixed deposit?")

In [ ]:
ask_question("how many types of saving accounts are there in a general bank except HLB?")